<img src="images/header-transparent.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de una playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 6 - Servicio de recomendación de playlists</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
    * [1.1 - Informática sin servidor](#section11)
    * [1.2 - Recursos empleados](#section12)
* [2. Base de datos](#section2)
    * [2.1 - Azure Cosmos DB](#section21)
    * [2.2 - Estructura](#section22)
    * [2.3 - Inserción de datos](#section23)
* [3. Aplicación de funciones](#section3)
* [4. API REST](#section4)
* [5. Aplicación Web](#section5)

<br>

---

In [ ]:
import csv
import datetime
import joblib
import os

from bson.timestamp import Timestamp
from bson.objectid import ObjectId
from collections import defaultdict
from pymongo import MongoClient


MPD_CSV_PATH = "MPD_CSV"
MODEL_DATA_FILE ="model/model_data.pkl"
GENRES_FILE = os.path.join(MPD_CSV_PATH, "mpd.genres.csv")
ARTISTS_FILE = os.path.join(MPD_CSV_PATH, "mpd.artists.csv")
ARTISTS_GENRES_FILE = os.path.join(MPD_CSV_PATH, "mpd.artists-genres.csv")
ALBUMS_FILE = os.path.join(MPD_CSV_PATH, "mpd.albums.csv")
TRACKS_FILE = os.path.join(MPD_CSV_PATH, "mpd.tracks.csv")
PLSTRS_FILE = os.path.join(MPD_CSV_PATH, "mpd.pls-tracks.csv")
PLS_INFO_FILE = os.path.join(MPD_CSV_PATH, "mpd.playlists-info.csv")

LOCAL_CONNECTION_STRING = "mongodb://localhost:27017/?readPreference=primary&directConnection=true&ssl=false"

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Una vez que ya tenemos disponible el punto de acceso con el modelo encargado de realizar las predicciones, vamos a montar el servicio web para la recomendación de playlists y la aplicación web que va a hacer uso de él.

<br>

---

<br>

<a id="section11"></a>

### <font color="#92002A">1.1 - Informática sin servidor</font>

<br>

Antes de empezar a desarrollar los servicios necesarios y mostrar qué tecnología que hemos empleado, vamos a hacer una breve introducción al concepto informática sin servidor para poder comprender los servicios usados para implementar el servicio de recomendación.

La informática sin servidor, o <i>serverless computing</i>, permite a los desarrolladores crear aplicaciones de forma rápida, ya que no es necesario que administren la infraestructura. Con las aplicaciones sin servidor, el proveedor de servicios en la nube aprovisiona, escala y administra automáticamente la infraestructura necesaria para ejecutar el código \cite{serverless}.

Para entender la definición de la informática sin servidor, es importante tener en cuenta que los servidores siguen ejecutando el código. El término sin servidor significa que las tareas asociadas con el aprovisionamiento y la administración de la infraestructura son invisibles para el desarrollador. Este enfoque permite a los desarrolladores centrarse más en la lógica de negocios y en aportar más valor al núcleo principal del negocio. La informática sin servidor ayuda a los equipos a aumentar su productividad y a comercializar los productos más rápido, además de permitir a las organizaciones optimizar mejor los recursos y seguir centrándose en la innovación.

Las principales ventajas que nos ofrece este modelo de servicios en la nube son las siguientes:

* <strong>No es necesaria la administración de la infraestructura</strong>: El uso de servicios totalmente administrados, permite a los desarrolladores evitar las tareas administrativas y centrarse en la lógica de negocios principal.
* <strong>Escalabilidad dinámica</strong>: Con la informática sin servidor, la infraestructura se escala y reduce verticalmente de forma dinámica en cuestión de segundos, a fin de satisfacer la demanda de cualquier carga de trabajo.
* <strong>Comercialización más rápida</strong>: Las aplicaciones sin servidor reducen las dependencias de las operaciones de cada ciclo de desarrollo, con el aumento de la agilidad de los equipos de desarrollo para ofrecer más funcionalidad en menos tiempo.
* <strong>Uso eficaz de los recursos</strong>: La migración a las tecnologías sin servidor ayuda a las organizaciones a reducir el coste total de propiedad y a reasignar los recursos para acelerar el ritmo de la innovación.

<br>

<a id="section12"></a>

### <font color="#92002A">1.2 - Recursos empleados</font>

<br>

Los recursos <i>serverless</i> que necesitamos para desarrollar nuestro servicio de recomendación de playlists son los siguientes:
* <i>Base de datos</i> (Azure Cosmos DB).
* <i>Aplicación de funciones</i> (Azure Functions).
* <i>Servicio API REST</i> (API Management).
* <i>Aplicación web</i>, alojada en un "blob" (contenedor de archivos) dentro de nuestra cuenta de almacenamiento.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section2"></a>
## <font color="#92002A">2 - Base de datos</font>
<br>

El primer recurso que vamos a crear es la base de datos donde almacenaremos toda la información de la que disponemos (playlists, artistas, pistas, ...) junto a otra información relevante sobre nuestro modelo, como las características de las pistas y las playlists.

Primero crearemos la base de datos en un equipo propio e incorporaremos los datos. Una vez esté creada en el equipo local, realizaremos un volcado de datos para alojarlos en nuestra cuenta del servicio de base de datos creada en el proveedor <i>cloud</i>.

<br>

---

<br>

<a id="section21"></a>
### <font color="#92002A">2.1 - Azure Cosmos DB</font>

<br>

*Azure Cosmos DB* es un servicio de base de datos multimodelo distribuido con escalado horizontal, que nos permite distribuir datos de forma global sobre cualquier número de regiones de *Microsoft Azure*, empleando un proceso transparente de escalado y replicación de los datos. También garantiza valores de latencia inferiores a 10 milisegundos en cualquier parte del mundo, además ofrece varios modelos de coherencia bien definidos para ajustar el rendimiento y garantizar alta disponibilidad con hospedaje múltiple.

<center><img src="images/AzureCosmosDB.png" width=400 alt="Logo CosmosDB"></center>

A nivel de funcionalidad, *Azure Cosmos DB* indexa datos automáticamente sin que haya que ocuparse de la administración de esquemas ni de índices. También se caracteriza por ser multimodelo y admitir de forma nativa varios modelo de datos:

* ***SQL API***: Documentos JSON y consultas basadas en SQL.
* ***Azure Table Storage***: Datos en formato clave-valor.
* ***MongoDB API***: Documentos JSON y uso de la API de *MongoDB*.
* ***Gremlin API***: Soporta la API de Gremlin 
* ***Casandra DB***: Para para representaciones de datos en columnas.  

<center><img src="images/FormatosCosmosDB.png" alt="Cosmos DB APIs"></center>

Para nuestra base de datos, vamos a emplear *Azure Cosmos DB* con la API de *MongoDB*. Este tipo de bases de datos se pueden utilizar como almacenes de datos para aplicaciones diseñadas para emplear *MongoDB*, de esta forma la aplicación se puede comunicar con *Azure Cosmos DB* y usar sus bases de datos en lugar de bases de datos de *MongoDB*. Lo único que se debe cambiar es la cadena de conexión.[1](https://unpocodejava.com/2018/02/22/que-es-azure-cosmos-db/) [2](https://medium.com/globant/introduction-to-cosmos-db-8d106bb7207) [3](https://docs.microsoft.com/es-es/azure/cosmos-db/introduction)

<br>

---

<br>

<a id="section22"></a>
### <font color="#92002A">2.2 - Estructura</font>

<br>

*Azure Cosmos DB* es un servicio de base de datos multimodelo distribuido con escalado horizontal, que nos permite distribuir datos de forma global sobre cualquier número de regiones de *Microsoft Azure*, empleando un proceso transparente de escalado y replicación de los datos. También garantiza valores de latencia inferiores a 10 milisegundos en cualquier parte del mundo, además ofrece varios modelos de coherencia bien definidos para ajustar el rendimiento y garantizar alta disponibilidad con hospedaje múltiple.

Para nuestro sistema de recomendación, vamos a crear una base de datos llamada *music* que contendrá las siguientes colecciones:
* <i>albums</i>: Contiene la información básica de los álbumes a los que pertenecen las pistas de las playlists.
* <i>artists</i>: Contiene todos los intérpretes a los cuales pertenecen los álbumes, así como a los géneros a los que pertenecen.
* <i>genres</i>: Contiene todos los géneros en los cuales se pueden clasificar los artistas.
* <i>playlists</i>: Contiene toda la información relativa a las listas de canciones, tales como el número de seguidores, si es colaborativa, última modificación, etc.
* <i>playlistsRecomm</i>: Esta colección la emplearemos para almacenar, de forma temporal, las playlists que recomiende nuestro sistema cuando solamente se le indique el título y/o características. Hemos creado esta colección para reducir el tiempo de espera y la carga al punto de conexión del modelo.
* <i>playlistFeatures</i>: Contiene las características/etiquetas con las que se pueden asociar las playlists.
* <i>tracks</i>: Colección con todas las pistas que existen en las playlists del sistema.
* <i>emojiTranslations</i>: Contiene el conjunto de emoticonos que pueden ser traducidos a texto, empleado para normalizar los títulos de las playlists y crear sus correspondientes etiquetas (playlist features).

<br>

A continuación, se muestra una ilustración con la arquitectura de la base de datos que hemos diseñado:

<img src="images/db_diagram.png" alt="Esquema de la BD" align="right">

<br>

---

<br>

<a id="section23"></a>
### <font color="#92002A">2.3 - Inserción de datos</font>

<br>

Antes de comenzar con la inserción de datos, creamos el cliente de *MongoDB* con la cadena de conexión a nuestra cuenta de *Azure Cosmos DB* y establecemos la base de datos a emplear (*music*):

In [ ]:
client = MongoClient(LOCAL_CONNECTION_STRING)
music_db = client['music']

<br>

Llegados a este punto, procedemos a cargar la información que tenemos repartida por los distintos ficheros *CSV*, transformarla en caso de ser necesario, e insertarla en las colecciones correspondientes de nuestra base de datos.

<br>

#### <font color="#92002A">Colección 'genres'</font>

<br>

In [ ]:
with open(GENRES_FILE, encoding="utf8") as file:
    genre_list = [{k: v for k, v in row.items()}
                   for row in csv.DictReader(file)]
    genre_list = [{'pid' : int(genre['genre_pid']), 'name' : genre['genre_name']} for genre in genre_list]
    
# Creamos un diccionario para establecer los identificadores de los géneros en los artistas
genres_map_dict = {genre['name'] : genre['pid'] for genre in genre_list}

In [ ]:
genres_col = music_db['genres']
genres_col.create_index('pid')

In [ ]:
genres_col.insert_many(genre_list, ordered=False)

In [ ]:
# Puesto que estamos cargando los datos a la base de datos de nuestro equipo,
# eliminaremos aquellos objetos que ya no necesitamos para evitar quedarnos sin espacio.
del genre_list

<br>

#### <font color="#92002A">Colección 'artists'</font>

<br>

In [ ]:
with open(ARTISTS_FILE, encoding="utf8") as file:
    artist_list = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]
    artist_list = [{'pid' : int(artist['artist_pid']), 
                    'name' : artist['artist_name'], 
                    '_id' : artist['artist_id']} for artist in artist_list]

In [ ]:
with open(ARTISTS_GENRES_FILE, encoding="utf8") as file:
    artist_genres = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]
    artist_genres = {artist['artist_id'] : [genres_map_dict[genre] 
                                         for genre in  artist['genres'].split('|') 
                                         if genre in genres_map_dict] 
                     for artist in artist_genres}  

In [ ]:
for artist in artist_list:
    if artist['_id'] in artist_genres:
        artist['genres'] = [genre for genre in artist_genres[artist['_id']]]

In [ ]:
artists_col = music_db['artists']
artists_col.create_index('pid')

In [ ]:
artists_col.insert_many(artist_list, ordered=False)

In [ ]:
del artist_list

<br>

#### <font color="#92002A">Colección 'albums'</font>

<br>

In [ ]:
with open(ALBUMS_FILE, encoding="utf8") as file:
    album_list = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]    
    album_list = [{'pid' : int(album['album_pid']),
                    'name' : album['album_name'],
                    '_id' : album['album_id'],
                    'artist' : album['artist_id']} for album in album_list]

In [ ]:
def date_converter(date_str):
    split = date_str.split('-')
    try:
        if len(split) == 3:
            return datetime.datetime(int(split[0]),int(split[1]), int(split[2]))
        elif len(split) == 2:
            return datetime.datetime(int(split[0]), int(split[1]), 1)
        else:
            return datetime.datetime(int(date_str),1,1)
    except:
        return None

In [ ]:
with open("MPD_CSV/mpd.albums-releasedate.csv", encoding="utf8") as file:
    dates_dict = {d['album_id'] : date_converter(d['release_date'])
                  for d in [{k: v for k, v in row.items()} for row in csv.DictReader(file)]
                  if d['release_date'] != None}

In [ ]:
for album in album_list:
    if album['_id'] in dates_dict:
        album['release_date'] = dates_dict[album['_id']]

del dates_dict

In [ ]:
albums_col = music_db['albums']
albums_col.create_index('pid')

albums_col.insert_many(album_list, ordered=False)

In [ ]:
del album_list

<br>

#### <font color="#92002A">Colección 'tracks'</font>

<br>

In [ ]:
with open(TRACKS_FILE, encoding="utf8") as file:
    track_list = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]    
    track_list = [{'pid' : int(track['track_pid']),
                    'name' : track['track_name'],
                    'duration_ms' : track['duration_ms'],
                    '_id' : track['track_id'],
                    'album' : track['album_id'],
                    'artist' : track['artist_id']} for track in track_list]
    
track_map_dict = {tr['pid'] : tr['_id'] for tr in track_list}

In [ ]:
tracks_col = music_db['tracks']
tracks_col.create_index('pid')

tracks_col.insert_many(track_list, ordered=False)

In [ ]:
del track_list

<br>

#### <font color="#92002A">Colección 'playlists'</font>

<br>

In [ ]:
plstrs_list_dict = defaultdict(list)

with open(PLSTRS_FILE) as file:
    file.readline() # Ignoramos el encabezado del CSV
    while (line := file.readline().rstrip()):
        pl_pid, _ , track_pid = line.split(',')
        plstrs_list_dict[int(pl_pid)].append(track_map_dict[int(track_pid)])

In [ ]:
pls_feats = joblib.load("model/model_data.pkl")['playlist_features']
pls_feats_names = joblib.load("model/model_data.pkl")['playlist_features_names']

pls_feats_dict = defaultdict(list)

for row,column,value in zip(pls_feats.row, pls_feats.col, pls_feats.data):
    if value != 0:
        feat = pls_feats_names[column]
        if 'name:' not in feat:
            pls_feats_dict[row].append(pls_feats_names[column].split(':')[-1])

In [ ]:
with open(PLS_INFO_FILE, encoding="utf8") as file:
    playlist_list = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]
    playlist_list = [{'pid' : int(playlist['pl_pid']),
                       'name' : playlist['name'],
                       'collaborative' : bool(playlist['collaborative']),
                       'tracksCount' : int(playlist['num_tracks']),
                       'albumsCount' : int(playlist['num_albums']),
                       'artistsCount' : int(playlist['num_artists']),
                       'followers' :  int(playlist['num_followers']),
                       'durationMs' : int(playlist['duration_ms']),
                       'editsCount' : int(playlist['num_edits']),
                       'modifiedAt' : Timestamp(int(playlist['modified_at']), 1),
                       'description' : playlist['description'],
                       'tracks' : plstrs_list_dict.pop(int(playlist['pl_pid']))} for playlist in playlist_list]

In [ ]:
for pl in playlist_list:
    if pl['pid'] in pls_feats_dict.keys():
        pl['tags'] = pls_feats_dict[pl['pid']]

In [ ]:
del track_map_dict
del pls_feats
del plstrs_list_dict
del pls_feats_dict

In [ ]:
known_pls_col = music_db['playlists']
known_pls_col.create_index('pid')

In [ ]:
known_pls_col.insert_many(playlist_list, ordered=False)

In [ ]:
del playlist_list

<br>

#### <font color="#92002A">Colección 'playlistFeatures'</font>

<br>

In [ ]:
pls_features = [{'pid' : i, 
                 'name' : x.split(':')[-1], 
                 'value' : x} for i,x in enumerate(pls_feats_names) 
                if "name:" not in x]

In [ ]:
plsfeat_col = music_db['playlistFeatures']
plsfeat_col.create_index('pid')

plsfeat_col.insert_many(pls_features, ordered=False)

In [ ]:
del pls_features

<br>

#### <font color="#92002A">Colección 'emojiTranslations'</font>

<br>

In [ ]:
with open("MPD_CSV/mpd.emoji-tags.csv", encoding="utf8") as file:
    emojis_list = [{k: v for k, v in row.items()}
                    for row in csv.DictReader(file)]
    emojis_list = [{'emoji' : e['emoji'], 'tags' : e['tags'].split('|')} for e in emojis_list]

In [ ]:
emojis_col = music_db['emojiTranslations']
emojis_col.insert_many(emojis_list, ordered=False)

In [ ]:
del emojis_list

---

<br>

Una vez creada la base de datos y sus colecciones, realizamos una serie de consultas y exportamos la base de datos a nuestra cuenta de <i>Azure Cosmos DB</i>. Este proceso lo vamos a realizar de la siguiente manera:
* Realizamos un volcado con la herramienta `mongodump`
* Mediante `mongorestore`, añadimos la base de datos y sus colecciones a <i>Azure Cosmos DB</i>.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section3"></a>
## <font color="#92002A">3 - Aplicación de funciones</font>

<br>

Una aplicación de funciones de <i>Azure Functions</i>, es una solución del tipo función como servicio o <i>FaaS</i> (<i>Function as a Service</i>), que nos permite ejecutar fácilmente pequeñas piezas de código, conocidas como <i>funciones</i>, en la nube. Podemos escribir únicamente el código necesario para realizar la funcionalidad que deseemos, sin tener que preocuparnos de una aplicación completa o de la infraestructura para ejecutarla. Estas funciones utilizan distintos lenguajes de desarrollo, como C#, Python, Java o Javascript, entre otros.

El coste que se nos factura es el tiempo de ejecución del código de nuestra función. Además, si tenemos muchas peticiones a nuestra función, <i>Microsoft Azure</i> escalará de forma elástica a más instancias o menos en función de la cantidad de peticiones que nuestra función tenga que atender.

En la figura que se muestra a continuación:

<img src="images/AzureFunctionsDiagram.png" alt="Azure Functions" align="right">

podemos ver el esquema de funcionamiento de las funciones. Un desencadenador, o <i>trigger</i>, hace que la función se ejecute con la entrada que se le proporciona, ya esté indicada en el <i>trigger</i> o enlazada en el código de dicha función. A continuación, se ejecutará con los datos proporcionados y producirá una salida, siendo del tipo admitido por el sistema (respuesta <i>HTTP</i>, fichero a almacenar en un <i>blob</i>, documento a insertar en una base de datos, etc.).

Para nuestro proyecto, se han definido una serie de funciones con tareas específicas para la recomendación de playlists, búsqueda de información en la base de datos, o interacción con <i>Spotify</i> (como la obtención de las playlists del usuario o la incorporación de las recomendadas por nuestro sistema). A continuación se detallan las funciones desarrolladas:
* <strong>GetEquivalentPlaylist</strong>: Busca una playlist, a partir de otra, que sea igual o similar. Consideramos que es igual si contiene las mismas canciones y el título, convertido a etiquetas, es el mismo. En caso de no encontrar ninguna igual, busca que las etiquetas creadas con el título y/o las canciones que contiene sean un subconjunto de otra playlist.
* <strong>GetPlaylistRecs</strong>: Mediante el identificador que tiene una playlist en la base de datos, devuelve un conjunto de playlist que recomienda el sistema.
* <strong>GetPlaylistNewRecs</strong>: A partir de un título, lo normaliza para convertirlo en etiquetas y llama al \textit{endpoint} del modelo de recomendación para obtener una lista de canciones. Una vez obtenidas las canciones, se seleccionan algunas de forma aleatoria y se crea una playlist con el resultado.
* <strong>GetPlaylistInfo</strong>: Función que se encarga de completar una playlist, cuya lista de pistas es un conjunto de identificadores, añadiendo información de la pista, del álbum y del artista al que pertenece.
* <strong>GetPlaylistInfo</strong>: Igual que la función anterior, pero se emplea en el caso de obtener varias playlists sobre las que se requiere completar la información.
* <strong>PostNewPlaylist</strong>: Mediante una playlist, identificador de usuario y código de autorización temporal, crea dicha playlist en la cuenta de <i>Spotify</i> del usuario indicado.
* <strong>GetUserPlaylist</strong>: Obtiene la playlist indicada de la colección del usuario, ya sea pública o privada.
* <strong>GetUserPlaylists</strong>: Igual que el caso anterior, pero para obtener varias playlists.

<br>

Podemos encontrar el proyecto de funciones en la carpeta `functions` 

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section4"></a>
## <font color="#92002A">4 - API REST</font>

<br>

A modo de recordatorio, una <i>API REST</i> es la implementación de una interfaz de programación de aplicaciones (<i>API</i>) que se adhiere a las restricciones de la arquitectura <i>REST</i>, actuando como interfaz. La comunicación entre el cliente y el servidor se realiza a través de las metodologías <i>HTTP</i>. En la siguiente figura podemos ver un esquema de funcionamiento:

<img src="images/rest-api-model.png" alt="Modelo API REST" align="center"/>

En nuestro caso, agruparemos las funciones que hemos definido en nuestro proyecto de <i>Azure Functions</i> como una <i>API REST</i> mediante <i>API Managment</i>, un espacio de trabajo que nos proporciona <i>Azure</i> en el que, básicamente, podemos configurar distintas <i>APIs</i> (propias o de terceros) añadiendo una capa de abstracción que unifica el acceso de desarrolladores a ellas.

<i>Azure API Management</i> admite la importación de instancias de <i>Azure Functions</i> como nuevas <i>APIs</i> o anexionándolas a un proyecto existente. El proceso genera automáticamente una clave de host en la aplicación de funciones y le asigna un valor con nombre en <i>API Management</i>.

Los <i>endpoints</i> que se han obtenido tras crear la <i>API REST</i> empleando el proyecto de <i>Azure Functions</i> en <i>API Managment</i>, son los siguientes:

* <strong>/recomm</strong>: Realiza recomendaciones para playlists según su <i>id</i> (está en el sistema) o según su lista de canciones y/o título. 
    * <strong><font color="blue">GET</font></strong> /recomm/playlists/\{id\}
    * <strong><font color="blue">GET</font></strong> /recomm/playlists
* <strong>/playlists</strong>: Se encarga de obtener playlists que se encuentran en nuestro sistema según su <i>id</i>, título, lista de canciones, etc. También es capaz de localizar una playlist lo más similar posible a una dada, para posteriormente realizar predicciones o buscar recomendaciones.
    * <strong><font color="blue">GET</font></strong> /playlists
    * <strong><font color="blue">GET</font></strong> /playlists/\{id\}
    * <strong><font color="blue">GET</font></strong> /playlists/equivalent
* <strong>/spotify</strong>: Empleado para obtener playlists del usuario en <i>Spotify</i> y para añadir las playlists que ha recomendado nuestro sistema en caso de que el usuario lo indique.
    * <strong><font color="blue">GET</font></strong> /spotify/playlists
    * <strong><font color="blue">GET</font></strong> /spotify/playlists/\{id\}
    * <strong><font color="green">POST</font></strong> /spotify/playlists

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section5"></a>
## <font color="#92002A">5 - Aplicación Web</font>

<br>

Para el desarrollo de la aplicación web, que empleara el servicio de recomendación que hemos creado, vamos a usar <i>Blazor</i>. 

<i>Blazor</i> es un proyecto desarrollado por <i>Microsoft</i> creado para permitir crear <i>SPAs</i> (<i>Single Page Applications</i>) únicamente usando como lenguajes de programación C# y <i>Razor Pages</i>, haciendo nula la necesidad de programar en Javascript o frameworks derivados.

La aplicación web que hemos desarrollado, la cual hace uso del servicio de recomendación que hemos creado, nos permite ver algunas de las funciones que se pueden implementar mediante el uso del modelo de recomendación.

En la pantalla inicial de la aplicación, como se muestra en la siguiente figura:

<img src="images/SerendipityApp1.png" alt="Página inicial" align="center"/>

avisamos al usuario del servicio que debe iniciar sesión en su cuenta de <i>Spotify</i> y autorizarla para leer sus playlists y crear nuevas listas.

Nuestra aplicación web realizara recomendaciones de playlists para usuarios de <i>Spotify</i> mediante dos formas:
* <strong>Mis playlists</strong>: En esta primera forma para la recomendación de playlists, accedemos a la cuenta <i>Spotify</i> del usuario y obtenemos las playlists de su colección. Una vez obtenidas, el usuario selecciona aquella sobre la cual quiere obtener recomendaciones y se le ofrecen varias playlists, pudiendo elegir cualquiera de ellas y almacenarla en su cuenta. En la siguiente figura, se muestra un ejemplo de esta opción:

<img src="images/SerendipityApp2.png" alt="Página Mis Playlists" align="center"/>

* <strong>¡Voy a tener suerte!</strong>: Ofrecemos al usuario la posibilidad de crear una playlist únicamente indicando un título. De esta forma podemos ver el redimiendo de nuestro modelo en aquellos casos de los que no se dispone de pistas (caso del <i>arranque en frío</i>. En la siguiente figura se muestra el resultado de la consulta <i>Summer Hits</i>:

<img src="images/SerendipityApp3.png" alt="Página ¡Voy a tener suerte!" align="center"/>

<br>

Podemos encontrar el proyecto de la aplicación web en la carpeta `webapp`.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>